In [1]:
from vllm import LLM, SamplingParams, EngineArgs, LLMEngine
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import torch
from vllm.usage.usage_lib import UsageContext
from typing import ClassVar, List, Optional, Sequence, Union, cast, overload

from vllm.engine.arg_utils import EngineArgs
from vllm.engine.llm_engine import LLMEngine
from vllm.inputs import PromptStrictInputs
from vllm.sampling_params import SamplingParams
from vllm.usage.usage_lib import UsageContext
from vllm.model_executor.layers.quantization.gptq import GPTQConfig
from transformers import BitsAndBytesConfig
from datetime import datetime

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# from vllm.model_executor.layers.quantization.bitsandbytes import BitsAndBytesConfig

In [18]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
conversation=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "What is the capital of Spain?"},]

In [23]:
tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    )[0]

tensor([128000, 128006,    882, 128007,    271,   3923,    374,    279,   6864,
           315,   9822,     30, 128009, 128006,  78191, 128007,    271,    791,
          6864,    315,   9822,    374,  12366,     13, 128009, 128006,    882,
        128007,    271,   3923,    374,    279,   6864,    315,  18157,     30,
        128009, 128006,  78191, 128007,    271])

In [24]:
print(tokenizer.batch_decode(
    tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    )[0]
))

['<|begin_of_text|>', '<|start_header_id|>', 'user', '<|end_header_id|>', '\n\n', 'What', ' is', ' the', ' capital', ' of', ' France', '?', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', '\n\n', 'The', ' capital', ' of', ' France', ' is', ' Paris', '.', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', '\n\n', 'What', ' is', ' the', ' capital', ' of', ' Spain', '?', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', '\n\n']


In [9]:

tokenizer.batch_decode(tokenizer([x['text'] for x in conversation])['input_ids'])

['<|begin_of_text|>What is the capital of France?',
 '<|begin_of_text|>The capital of France is Paris.',
 '<|begin_of_text|>What is the capital of Spain?']

In [3]:
prompts_string = [
        {
            "role": "system",
            "content": "You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most "
        },
        {
            "role": "user",
            "content": "import abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal[\"dontcare\", \"cheap\", \"moderate\", \"expensive\"]\nHotelType = Literal[\"hotel\", \"guest house\", \"dontcare\"]\nOption = Literal[\"yes\", \"no\", \"dontcare\"]\nDayOfWeek = Literal[\"monday\", \"tuesday\", \"wednesday\", \"thursday\", \"friday\", \"saturday\", \"sunday\"]\nArea = Literal[\"dontcare\", \"centre\", \"east\", \"north\", \"south\", \"west\"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal[\"dontcare\"]] = None  # between 0 and 5 or dontcare\n    internet: Option = None\n\n\n@dataclass\nclass Train:\n    destination: str = None\n    leave_from: str = None\n    day: DayOfWeek = None\n    book_people: int = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\nAttractionType = Literal[\"architecture\", \"boat\", \"church\", \"cinema\", \"college\", \"concert hall\", \"entertainment\",\n                         \"hotspot\", \"multiple sports\", \"museum\", \"nightclub\", \"park\", \"special\", \"swimming pool\",\n                         \"theatre\", \"dontcare\"]\n\n\n@dataclass\nclass Attraction:\n    name: str = None\n    area: Area = None\n    type: AttractionType = None\n\n\n@dataclass\nclass Restaurant:\n    name: str = None\n    food_type: str = None\n    price_range: PriceRange = None\n    area: Area = None\n    book_time: str = None  # hh:mm or dontcare\n    book_day: DayOfWeek = None\n    book_people: int = None\n\n\n@dataclass\nclass Taxi:\n    destination: str = None\n    leave_from: str = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\n@dataclass\nclass BeliefState:\n    hotel: Hotel = None\n    train: Train = None\n    attraction: Attraction = None\n    restaurant: Restaurant = None\n    taxi: Taxi = None\n\n\nclass DialogueAgent(abc.ABC):\n\n    state: BeliefState\n\n    @abc.abstractmethod\n    def find_hotel(self, name: str = None, price_range: PriceRange = None, type: HotelType = None,\n                    parking: Option = None, book_number_of_days: int = None, book_day: DayOfWeek = None,\n                    book_people: int = None, area: Area = None, stars: Union[int, Literal[\"dontcare\"]] = None,\n                    internet: Option = None) -> Hotel:\n        pass\n\n    @abc.abstractmethod\n    def find_train(self, destination: str = None, leave_from: str = None, day: DayOfWeek = None,\n                   book_people: int = None, depart_time: str = None, arrive_by_time: str = None) -> Train:\n        pass\n\n    @abc.abstractmethod\n    def find_attraction(self, name: str = None, area: Area = None, type: AttractionType = None) -> Attraction:\n        pass\n\n    @abc.abstractmethod\n    def find_restaurant(self, name: str = None, food_type: str = None, price_range: PriceRange = None,\n                        area: Area = None, book_time: str = None, book_day: DayOfWeek = None,\n                        book_people: int = None, ) -> Restaurant:\n        pass\n\n    @abc.abstractmethod\n    def find_taxi(self, destination: str = None, leave_from: str = None, depart_time: str = None,\n                  arrive_by_time: str = None) -> Taxi:\n        pass\n\n    def get_state(self) -> BeliefState:\n        return state\n\n\nif __name__ == '__main__':\n    agent = DialogueAgent()\n    state = BeliefState()\n\n    \n\n    #### Example 1 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}, \"attraction\": {\"type\": \"museum\", \"area\": \"centre\"}})\n    print(\"system: i have the express by holiday inn cambridge that matches your preferences . would you like me to book it for you ?\")\n    print(\"user: does the hotel include free parking ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 2 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"parking\": \"yes\", \"price_range\": \"expensive\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: i like the express by holiday inn cambridge . can i get you some more information on that 1 ?\")\n    print(\"user: no , let s just book it .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 3 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"parking\": \"yes\", \"stars\": 2, \"type\": \"hotel\"}})\n    print(\"system: the ashley hotel , lovell lodge and express by holiday inn cambridge match your criteria . would you like more info on them ?\")\n    print(\"user: i need a guest house .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(type=\"guest house\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 4 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: yes it does . would you like me to book it ?\")\n    print(\"user: no , thank you , that s all i need .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.update({})  # no change\n"}, 
        {
            "role": "user", "content": "\n    #### Example 5 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"price_range\": \"expensive\"}, \"attraction\": {\"name\": state.attraction.name}})\n    print(\"system: i have 1 listing for express by holiday inn cambridge , is that okay for you ?\")\n    print(\"user: that works fine .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 6 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"book_day\": \"saturday\", \"book_people\": 1, \"book_number_of_days\": 4, \"name\": state.hotel.name, \"area\": \"east\", \"stars\": 2, \"internet\": \"yes\", \"type\": \"hotel\"}, \"attraction\": {\"area\": \"south\"}, \"taxi\": {\"depart_time\": state.taxi.depart_time, \"leave_from\": state.taxi.leave_from}})\n    print(\"system: the south has cinema , entertainment , museum , nightclub , park and theatre . which do you prefer ?\")\n    print(\"user: can you make a recommendation ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.attraction = agent.find_attraction(type=\"dontcare\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 7 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}, \"restaurant\": {\"food_type\": state.restaurant.food_type, \"price_range\": \"moderate\", \"area\": \"east\"}})\n    print(\"system: okay the pizza hut fen ditton fits that criteria . it is located in cambridge retail park .\")\n    print(\"user: could you provide me with the address , phone number and postcode ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.restaurant = agent.find_restaurant(name=\"pizza hut fenditton\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 8 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: i have found express by holiday inn cambridge . 1517 norman way , coldhams business park\")\n    print(\"user: does that hotel include free parking and have 2 stars ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 9 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"price_range\": \"expensive\"}, \"attraction\": {\"name\": state.attraction.name}})\n    print(\"system: alright . they are at 01223866800 at the property is on 1517 norman way , coldhams business park . would you like a reservation ?\")\n    print(\"user: what s the star rating ? i want to stay at a 4 star place .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(stars=\"4\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 10 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"dontcare\", \"price_range\": \"expensive\", \"stars\": 2}})\n    print(\"system: okay , the express by holiday inn cambridge matches the criteria . the address is 1517 norman way , coldhams business park . would you like me to book a reservation ?\")\n    print(\"user: does it have free parking\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\", parking=\"yes\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 11 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name}})\n    print(\"system: i have the express by holiday inn cambridge located on 1517 norman way , coldhams business park . their phone number is 01223866800 . would you like to know anything else ?\")\n    print(\"user: yes , could you book the hotel room for me for 7 people ?\")\n"},
        
    ]

In [4]:
# model = LLM(model="/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-AWQ", quantization='awq', enforce_eager=True)
 
model = LLM(model="/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-GPTQ", quantization='gptq', enforce_eager=False)

# model = LLM(model="/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-EXL2", enforce_eager=False)

INFO 07-24 10:30:16 gptq_marlin.py:91] Detected that the model can run with gptq_marlin, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_marlin for faster inference
WARNING 07-24 10:30:16 config.py:241] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-24 10:30:16 llm_engine.py:174] Initializing an LLM engine (v0.5.2) with config: model='/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-GPTQ', speculative_config=None, tokenizer='/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-24 10:33:23 model_runner.py:266] Loading model weights took 37.0657 GB
INFO 07-24 10:33:31 gpu_executor.py:86] # GPU blocks: 6445, # CPU blocks: 819
INFO 07-24 10:33:36 model_runner.py:1007] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-24 10:33:36 model_runner.py:1011] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-24 10:34:02 model_runner.py:1208] Graph capturing finished in 26 secs.


In [5]:
tokenizer = model.get_tokenizer()
stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
stop_sequence = ['--',  ';', '#', '\n']
samplig_params = SamplingParams(n=1, best_of=1, max_tokens=120, temperature=0, stop=stop_sequence,stop_token_ids=stop_token_ids)

prompt_token_ids = tokenizer.apply_chat_template(
                    prompts_string,
                    add_generation_prompt=True,
                    return_tensors='pt').int()
prompts = tokenizer.batch_decode(prompt_token_ids)
# inputs = cast(Union[PromptStrictInputs, Sequence[PromptStrictInputs]],
#             prompts)

# start = datetime.now()
# outputs = model.generate(prompts, sampling_params=samplig_params)
# running_secs = (datetime.now() - start).seconds
# print(outputs[0].outputs[0].text)
# print(running_secs)

In [6]:
outputs = model.generate(prompts, sampling_params=samplig_params)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.23s/it, est. speed input: 712.96 toks/s, output: 7.44 toks/s]


In [8]:
outputs[0].outputs[0].text

'agent.state.hotel = agent.find_hotel(name="express by holiday inn cambridge", book_people=7)'

In [9]:
prompts_10 = prompts*10
print(len(prompts_10))
print()

start = datetime.now()
for _ in range(3):
    for _ in range(3):
        outputs = model.generate(prompts_10, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

10



Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it, est. speed input: 929.49 toks/s, output: 9.70 toks/s]

192


In [ ]:
prompts_30 = prompts * 30
print(len(prompts_30))
print()

start = datetime.now()
for _ in range(3):
    outputs = model.generate(prompts_30, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

In [ ]:
prompts_30_3 = prompts_30 * 3
print(len(prompts_30_3))
print()
start = datetime.now()
outputs = model.generate(prompts_30_3, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

Processed prompts: 100%|██████████| 90/90 [03:13<00:00,  2.15s/it, est. speed input: 1067.71 toks/s, output: 11.15 toks/s]

194


In [1]:
import sys, os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from exllamav2 import ExLlamaV2, ExLlamaV2Config, ExLlamaV2Cache, ExLlamaV2Tokenizer, Timer
from exllamav2.generator import ExLlamaV2DynamicGenerator

model_dir = "/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-EXL2"
config = ExLlamaV2Config(model_dir)
config.arch_compat_overrides()
model = ExLlamaV2(config)
cache = ExLlamaV2Cache(model, max_seq_len = 32768, lazy = True)
model.load_autosplit(cache, progress = True)

print("Loading tokenizer...")
tokenizer = ExLlamaV2Tokenizer(config)

# Initialize the generator with all default parameters

generator = ExLlamaV2DynamicGenerator(
    model = model,
    cache = cache,
    tokenizer = tokenizer,
)

/home/haesungpyun/anaconda3/envs/torch2.1_clone/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Output()

Loading tokenizer...


In [2]:
from exllamav2.generator import ExLlamaV2Sampler

settings = ExLlamaV2Sampler.Settings(
    temperature = 0,
    top_k = 1,
    top_p = 0,
    top_a = 0,
    typical = 0,
    skew = 0,
    token_repetition_penalty = 1.0,
    token_frequency_penalty = 0.0,
    token_presence_penalty = 0.0,
    smoothing_factor = 0.0,
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
prompts = tokenizer.batch_decode(tokenizer.apply_chat_template(prompts_string, add_generation_prompt=True, return_tensors='pt'))
prompts

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nimport abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]\nHotelType = Literal["hotel", "guest house", "dontcare"]\nOption = Literal["yes", "no", "dontcare"]\nDayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]\nArea = Literal["dontcare", "centre", "east", "north", "south", "west"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal["d

In [ ]:
stop_token_ids = [tokenizer.eos_token_id, tokenizer.encode("<|eot_id|>").item()]
stop_sequence = ['--',  ';', '#', '\n']
# samplig_params = SamplingParams(n=1, best_of=1, max_tokens=120, temperature=0, stop=stop_sequence,stop_token_ids=stop_token_ids)

# prompt_token_ids = tokenizer.apply_chat_template(
#                     prompts_string,
#                     add_generation_prompt=True,
#                     return_tensors='pt').int()
# prompts = tokenizer.batch_decode(prompt_token_ids)

In [10]:
from datetime import datetime

max_new_tokens = 120

# Warmup generator. The function runs a small completion job to allow all the kernels to fully initialize and
# autotune before we do any timing measurements. It can be a little slow for larger models and is not needed
# to produce correct output.

generator.warmup()

# Generate one completion, using default settings

with Timer() as t_single:
    start = datetime.now()
    output = generator.generate(prompt = prompts[0], max_new_tokens = max_new_tokens, add_bos = True)
    running_secs = (datetime.now() - start).seconds

print("-----------------------------------------------------------------------------------")
print("- Single completion")
print("Running Seconds : ", running_secs)
print("-----------------------------------------------------------------------------------")
print(output)
print()


-----------------------------------------------------------------------------------
- Single completion
Running Seconds :  6
-----------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. 
 Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>

import abc
from dataclasses import dataclass
from typing import Literal, Union

PriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]
HotelType = Literal["hotel", "guest house", "dontcare"]
Option = Literal["yes", "no", "dontcare"]
DayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
Area = Literal["dontcare", "centre", "east", "north", "south", "west"]


@dataclass
class Hotel:
    name: str = None
    price_range: PriceRange = None
    type: HotelType = 

In [14]:
prompts_10 = prompts*10
prompts_10
prompts_30 = prompts*30*3

In [15]:
with Timer() as t_batched:
    start = datetime.now()
    outputs = generator.generate(prompt = prompts_30, max_new_tokens = max_new_tokens, add_bos = True)
    running_secs = (datetime.now() - start).seconds

print('-----------------------------------------------------------------------------------')
print("Running Seconds: ", running_secs)
print('-----------------------------------------------------------------------------------')

for idx, output in enumerate(outputs):
    print("-----------------------------------------------------------------------------------")
    print(f"- Batched completion #{idx + 1}")
    print("-----------------------------------------------------------------------------------")
    print(output)
    print()

print("-----------------------------------------------------------------------------------")
print(f"speed, bsz 1: {max_new_tokens / t_single.interval:.2f} tokens/second")
print(f"speed, bsz {len(prompts)}: {max_new_tokens * len(prompts) / t_batched.interval:.2f} tokens/second")

-----------------------------------------------------------------------------------
Running Seconds:  118
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
- Batched completion #1
-----------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. 
 Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>

import abc
from dataclasses import dataclass
from typing import Literal, Union

PriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]
HotelType = Literal["hotel", "guest house", "dontcare"]
Option = Literal["yes", "no", "dontcare"]
DayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
Ar

In [16]:
running_secs

118

: 

In [3]:
engine='meta-llama/Meta-Llama-3-70B-Instruct'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = LlamaForCausalLM.from_pretrained(
    engine,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=False
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
messages_chat

tensor([[128000, 128006,   9125,  ...,  78191, 128007,    271]])

In [8]:
tokenizer = AutoTokenizer.from_pretrained(engine)
tokenizer.pad_token_id = tokenizer.eos_token_id

stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
stop_sequence = ['--',  ';', '#', '\n']

input_ids = tokenizer.apply_chat_template(
                    prompts_string,
                    add_generation_prompt=True,
                    return_tensors="pt")

sequences, logits = [], []

start = datetime.now()
result = model.generate(
    input_ids,
    max_new_tokens=120,
    eos_token_id=stop_token_ids,
    do_sample=True,
    top_p=0.9,
    output_scores= True,
    return_dict_in_generate=True,
    stop_strings=stop_sequence,
    tokenizer=tokenizer,
    output_logits=True,
    )
running_secs = (datetime.now() - start).seconds

sequences.append(result['sequences'][0])
logits.append(result['logits'])
print(sequences)
print(running_secs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/haesungpyun/anaconda3/envs/torch2.1_clone/lib/python3.10/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to 

[tensor([128000, 128006,   9125,  ...,     22,      8, 128009])]
11


In [11]:
len(sequences[0])/11

212.54545454545453

In [ ]:
engine='meta-llama/Meta-Llama-3-70B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(engine)
tokenizer.pad_token_id = tokenizer.eos_token_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = LlamaForCausalLM.from_pretrained(
    engine,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=False
)
# Define the directory where the model will be saved
save_directory = "/data1/home/haesungpyun/_8b_quantized"

# Save the model and tokenizer to the specified directory
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
llm = LLM(model='hacastle12/Meta-Llama-3-70B-Instruct-BitsAndBytes')

In [ ]:
engine_args = EngineArgs(
    model='hacastle12/Meta-Llama-3-70B-Instruct-BitsAndBytes',
    # quantization='bitsandbytes',
    # qlora_adapter_name_or_path='timdettmers/qlora-flan-7b',
    # load_format='bitsandbytes',
    # enable_lora=False,
    # max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
llm_engine = LLMEngine.from_engine_args(
            engine_args, usage_context=UsageContext.LLM_CLASS)
request_counter = Counter()


In [ ]:

inputs = cast(
        Union[PromptStrictInputs, Sequence[PromptStrictInputs]],
        messages)

In [ ]:
for idx, msg in enumerate(messages):
        llm_engine.add_request(
                request_id=str(idx),
                inputs={"prompt_token_ids":msg},
                params=samplig_params,
                lora_request=None
        )
outputs = []
while llm_engine.has_unfinished_requests():
    step_outputs = llm_engine.step()
    for output in step_outputs:
        if output.finished:
            outputs.append(output)
outputs = sorted(outputs, key=lambda x: int(x.request_id))

In [ ]:
outputs[0].outputs[0].text

In [ ]:
test_config= {
"name": "qlora_inference_example",
'model': "meta-llama/Meta-Llama-3-8B-Instruct",
'quantization': "bitsandbytes",
'lora_repo': 'timdettmers/qlora-flan-7b'
}

engine_args = EngineArgs(
    model='/data1/home/haesungpyun/llama-3-70b-instruct-awq',
    quantization='AWQ',
    # enable_lora=True,
    # max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
engine = LLMEngine.from_engine_args(engine_args)


# engine_args = EngineArgs(model='mcasperhansen/llama-3-8b-instruct-awq', quantization='AWQ', enforce_eager=True)

In [ ]:
tokenizer = engine.get_tokenizer()

In [ ]:
messages.__len__()

In [ ]:
test_prompts = [(messages[0], samplig_params, None)]
request_id = 0
while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(str(request_id),
                               prompt,
                               sampling_params,
                               lora_request=lora_request)
            request_id += 1

        request_outputs = engine.step()
        for request_output in request_outputs:
            if request_output.finished:
                print("----------------------------------------------------")
                print(f"Prompt: {request_output.prompt}")
                print(f"Output: {request_output.outputs[0].text}")

In [ ]:
engine.add_request(str(0), messages[0], samplig_params, lora_request=None)

In [ ]:
engine.has_unfinished_requests()

In [ ]:
while engine.has_unfinished_requests():
    output = engine.step()
    for o in output:
        print(o.finished)

In [ ]:
output[0].outputs[0].text

In [ ]:
"""
This example shows how to use LoRA with different quantization techniques
for offline inference.

Requires HuggingFace credentials for access.
"""

import gc
from typing import List, Optional, Tuple

import torch
from huggingface_hub import snapshot_download

from vllm import EngineArgs, LLMEngine, RequestOutput, SamplingParams
from vllm.lora.request import LoRARequest


"""Main function that sets up and runs the prompt processing."""
test_config= {
"name": "qlora_inference_example",
'model': "meta-llama/Meta-Llama-3-8B-Instruct",
'quantization': "bitsandbytes",
'lora_repo': 'timdettmers/qlora-flan-7b'
}

engine_args = EngineArgs(
    model=test_config['model'],
    quantization=test_config['quantization'],
    qlora_adapter_name_or_path=test_config['lora_repo'],
    load_format=test_config['quantization'],
    enable_lora=True,
    max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
engine = LLMEngine.from_engine_args(engine_args)

lora_path = snapshot_download(repo_id=test_config['lora_repo'])


In [ ]:
test_prompts = [
     ('Say Hello to me',SamplingParams(temperature=0.0,
                logprobs=1,
                prompt_logprobs=1,
                max_tokens=128), None),
    # ("my name is", SamplingParams(temperature=0.0,
    #             logprobs=1,
    #             prompt_logprobs=1,
    #             max_tokens=128),
    # LoRARequest("lora-test-1", 1, lora_path))
    ]
request_id = 0
while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(str(request_id),
                               prompt,
                               sampling_params,
                               lora_request=lora_request)
            request_id += 1

        request_outputs: List[RequestOutput] = engine.step()
        for request_output in request_outputs:
            if request_output.finished:
                print("----------------------------------------------------")
                print(f"Prompt: {request_output.prompt}")
                print(f"Output: {request_output.outputs[0].text}")

In [ ]:
request_outputs[-1].outputs

In [ ]:
model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
messages = [
            {
                "role": "system",
                "content": "You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most "
            },
            {
                "role": "user",
                "content": "import abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal[\"dontcare\", \"cheap\", \"moderate\", \"expensive\"]\nHotelType = Literal[\"hotel\", \"guest house\", \"dontcare\"]\nOption = Literal[\"yes\", \"no\", \"dontcare\"]\nDayOfWeek = Literal[\"monday\", \"tuesday\", \"wednesday\", \"thursday\", \"friday\", \"saturday\", \"sunday\"]\nArea = Literal[\"dontcare\", \"centre\", \"east\", \"north\", \"south\", \"west\"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal[\"dontcare\"]] = None  # between 0 and 5 or dontcare\n    internet: Option = None\n\n\n@dataclass\nclass Train:\n    destination: str = None\n    leave_from: str = None\n    day: DayOfWeek = None\n    book_people: int = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\nAttractionType = Literal[\"architecture\", \"boat\", \"church\", \"cinema\", \"college\", \"concert hall\", \"entertainment\",\n                         \"hotspot\", \"multiple sports\", \"museum\", \"nightclub\", \"park\", \"special\", \"swimming pool\",\n                         \"theatre\", \"dontcare\"]\n\n\n@dataclass\nclass Attraction:\n    name: str = None\n    area: Area = None\n    type: AttractionType = None\n\n\n@dataclass\nclass Restaurant:\n    name: str = None\n    food_type: str = None\n    price_range: PriceRange = None\n    area: Area = None\n    book_time: str = None  # hh:mm or dontcare\n    book_day: DayOfWeek = None\n    book_people: int = None\n\n\n@dataclass\nclass Taxi:\n    destination: str = None\n    leave_from: str = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\n@dataclass\nclass BeliefState:\n    hotel: Hotel = None\n    train: Train = None\n    attraction: Attraction = None\n    restaurant: Restaurant = None\n    taxi: Taxi = None\n\n\nclass DialogueAgent(abc.ABC):\n\n    state: BeliefState\n\n    @abc.abstractmethod\n    def find_hotel(self, name: str = None, price_range: PriceRange = None, type: HotelType = None,\n                    parking: Option = None, book_number_of_days: int = None, book_day: DayOfWeek = None,\n                    book_people: int = None, area: Area = None, stars: Union[int, Literal[\"dontcare\"]] = None,\n                    internet: Option = None) -> Hotel:\n        pass\n\n    @abc.abstractmethod\n    def find_train(self, destination: str = None, leave_from: str = None, day: DayOfWeek = None,\n                   book_people: int = None, depart_time: str = None, arrive_by_time: str = None) -> Train:\n        pass\n\n    @abc.abstractmethod\n    def find_attraction(self, name: str = None, area: Area = None, type: AttractionType = None) -> Attraction:\n        pass\n\n    @abc.abstractmethod\n    def find_restaurant(self, name: str = None, food_type: str = None, price_range: PriceRange = None,\n                        area: Area = None, book_time: str = None, book_day: DayOfWeek = None,\n                        book_people: int = None, ) -> Restaurant:\n        pass\n\n    @abc.abstractmethod\n    def find_taxi(self, destination: str = None, leave_from: str = None, depart_time: str = None,\n                  arrive_by_time: str = None) -> Taxi:\n        pass\n\n    def get_state(self) -> BeliefState:\n        return state\n\n\nif __name__ == '__main__':\n    agent = DialogueAgent()\n    state = BeliefState()\n\n    \n\n    #### Example 1 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"north\", \"parking\": \"yes\", \"price_range\": \"moderate\", \"stars\": 4, \"internet\": \"yes\"}})\n    print(\"system: there are 6 hotel -s in the north part of the city that fall in the moderate price range , have free wifi and parking , and have 4 stars . any other specification -s ?\")\n    print(\"user: i am not looking to book yet , can you tell me their price range -s ?\")\n"
            },
            {
                "role": "assistant",
                "content": "    agent.state.update({})  # no change\n"
            },
            {
                "role": "user",
                "content": "\n    #### Example 11 ####\n    agent.state = BeliefState.from_dict({})\n    print(\"user: i looking for information about a hotel in the moderate price range that include -s free wifi .\")\n"
            }
        ]


In [ ]:
stop_sequences = ['--', '\n', ';', '#']
terminators =  [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")    
]
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

sequences, logits = [], []
for _ in range(2):
    # result = openai.completions.create(**args)
    result = model.generate(
        input_ids,
        max_new_tokens=120,
        eos_token_id=terminators,
        do_sample=True,
        top_p=0.9,
        output_scores= True,
        return_dict_in_generate=True,
        stop_strings=stop_sequences,
        tokenizer=tokenizer,
        output_logits=True,
        )

    sequences.append(result['sequences'][0])
    logits.append(result['logits'])


In [ ]:
[len(logit) for logit in logits]

In [ ]:
[torch.stack(logit).softmax(dim=-1).max(dim=-1)[0].log().sum().item() for logit in logits]

In [ ]:
dict(zip(
    [tokenizer.decode(seq[input_len:], skip_special_tokens=True) for seq in sequences],
    [torch.stack(logit).softmax(dim=-1).max(dim=-1)[0].log().sum(0).item() for logit in logits]))

In [ ]:
result['sequences'].shape

In [ ]:
input_len = input_ids.shape[-1]

dict(
    zip(
        [tokenizer.decode(seq[input_len:], skip_special_tokens=True) for seq in result['sequences']] ,
        torch.stack(result['scores']).softmax(dim=-1).max(dim=-1)[0].log().sum(0)
        )
    )

In [ ]:
torch.stack(result['scores']).softmax(dim=-1).max(dim=-1)[0].log().sum(0)

In [ ]:
# list of torch.tensor to a tensor
from torch import nn
torch.max(nn.functional.softmax(torch.stack(result['logits']), dim=-1), dim=-1)[0].log().sum().item()

In [ ]:
torch.stack(result['logits']).softmax(dim=-1).max(dim=-1)[0].log().sum().item()

In [ ]:
from transformers import StoppingCriteria

stop_sequences =  [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")    
]
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

result_2 = model.generate(
    input_ids,
    max_new_tokens=120,
    eos_token_id=stop_sequences,
    stop_strings=["\n\n", "#", "print("],
    do_sample=False,
    temperature=0.0,
    output_scores= True,
    return_dict_in_generate=True,
    tokenizer=tokenizer
)
tokenizer.decode(result_2['sequences'][0][input_ids.shape[-1]:], skip_special_tokens=True)

In [ ]:
result_2['sequences'][0][input_ids.shape[-1]:].__len__()

In [ ]:
result_2['sequences'][0][input_ids.shape[-1]:]

In [ ]:
from torch import nn
a = 1
a *= max(nn.functional.softmax(logits) for logits in result_2['scores'])

In [ ]:
a = 1
for logits in result_2['scores']:
    a*=torch.max(nn.functional.softmax(logits, dim=-1))

In [ ]:
a

In [ ]:
result_2['sequences'], outputs[0][input_ids.shape[-1]:], response

In [ ]:
tokenizer.decode(result_2['sequences'][0][input_ids.shape[-1]:], skip_special_tokens=False), 

In [ ]:
result_2['logits']